Testing Propotype with Selenium & google library
*** I have using FireFox for the selenium web driver. Be sure that Firefox has been installed on the local machine.

In [ ]:
pip install selenium pandas google


In [16]:
#from googlesearch import search_news
from googlesearch import search
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from urllib.parse import urlparse
import pandas as pd
import time
import traceback


In [18]:
OUTPUT_FILE = 'output.csv'
AUTHOR_MAX_WORDS = 3
SLEEP_INTERVAL = 2

# Cuntion for Getting Date from Content
def extractDate(String):
    try:
        return False
    except:
        print("Error accessing a text:", e)
    return True

# Function for Getting Content from url
def getContent(driver,url):
    try:
        parsed_url = urlparse(url)

        authors = '' 
        source_domain = parsed_url.netloc
        copyright = ''
        paragraph = ''
        title = ''
        readMoreUrl = ''
        date = ''
        driver.get(url)
        H1_elements = driver.find_elements(By.TAG_NAME, "h1")
        p_elements = driver.find_elements(By.TAG_NAME, "p")
        a_elements = driver.find_elements(By.TAG_NAME, "a")

        # process with Readmore link
        for a in a_elements:
            try:
                value = a.text
                if value.casefold() == 'read more':
                    readMoreUrl = a.get_attribute('href')
            except Exception as e:
                print("Error accessing a text:", e)
                continue
            
        # Getting Title
        for h1 in H1_elements:
            try:
                title = h1.text
                break  # Assuming you only need the first h1's text
            except Exception as e:
                print("Error accessing h1 text:", e)
                continue

        # Getting Content
        for p in p_elements:
            try:
                value = p.text
                # Filter out authors
                if value.startswith('By '):
                    authors = value.replace('By ', '') 
                    words = authors.split()
                    if len(words) <= AUTHOR_MAX_WORDS and not authors:
                        authors = value
                    else:
                        authors = ''
                    continue
                # Filter out Copyright
                if '©' in value or 'All Rights Reserved' in value:
                    copyright = value
                    continue

                # call date function to extract the date
                if (extractDate(value)):
                    date = value
                    continue

                paragraph += value + ' '
            except Exception as e:
                print(f"Error accessing paragraph text from {url}:\n", e)
                continue
    except TimeoutException:
        authors = ''
        print(f"Timeout waiting for elements on the page: {url}")
    except NoSuchElementException:
        authors = ''
        print(f"An element was not found on the page: {url}")
    except Exception as e:
        authors = ''
        print(f"An unexpected error occurred with URL => {url}: {e}")

    return {'authors': authors, 'date' : date, 'title' : title, 'content': paragraph, 'source_domain': source_domain, 'url' : url, 'copyright': copyright, 'readMoreUrl' : readMoreUrl }

def save_to_csv_with_pandas(recordSet, filename):
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(recordSet)
    # Save the DataFrame to a CSV file
    df.to_csv(filename, index=False, encoding='utf-8')



In [17]:
# define query String
queryString = 'Did Joe Biden tell voters to stay home and not vote?'

#Search in google url with google library
results = search(query=queryString,num = 10 )

In [19]:
dataSet = []
driver = webdriver.Firefox()
i = 0
for url in results:
    print(f'Scripting from => {url}')
    try:
        dataSet.append(getContent(driver,url))
    except:
        print(f"An unexpected error occurred with URL => {url}: {e}")
    #time.sleep(SLEEP_INTERVAL)
driver.close()
df = pd.DataFrame(dataSet)
df

Scripting from => https://www.voanews.com/a/7450337.html
Scripting from => https://www.usnews.com/news/top-news/articles/2024-01-22/fake-biden-robo-call-tells-new-hampshire-voters-to-stay-home
Scripting from => https://www.dailyherald.com/20240128/news/biden-didnt-tell-voters-not-to-vote/
Scripting from => https://www.nbcnews.com/politics/2024-election/fake-joe-biden-robocall-tells-new-hampshire-democrats-not-vote-tuesday-rcna134984
Scripting from => https://www.reuters.com/article/uk-fact-check-biden-voter-protection-not-idUSKBN27E2VH/
Scripting from => https://www.politico.com/news/2020/03/13/democrats-confront-a-never-biden-contingent-127438
Scripting from => https://www.theguardian.com/us-news/video/2020/may/22/joe-biden-charlamagne-you-aint-black-trump-video
Scripting from => https://www.latimes.com/opinion/story/2020-05-22/biden-says-if-youre-black-and-dont-vote-for-him-youre-not-black-hes-right
Scripting from => https://www.pbs.org/newshour/politics/biden-plans-to-stay-home-test

,authors,date,title,content,source_domain,url,copyright,readMoreUrl
0,,,Fake Biden Robo-Call Tells New Hampshire Voter...,Print As New Hampshire voters prepared to cast...,www.voanews.com,https://www.voanews.com/a/7450337.html,,
1,,,Fake 'Biden' Robocall Tells New Hampshire Demo...,REUTERS FILE PHOTO: A potential voter ta...,www.usnews.com,https://www.usnews.com/news/top-news/articles/...,,
2,,,Biden didn’t tell voters not to vote,,www.dailyherald.com,https://www.dailyherald.com/20240128/news/bide...,,
3,,,Fake Joe Biden robocall tells New Hampshire De...,"MANCHESTER, N.H. — The New Hampshire a...",www.nbcnews.com,https://www.nbcnews.com/politics/2024-election...,© 2024 NBC UNIVERSAL,
4,,,,,www.reuters.com,https://www.reuters.com/article/uk-fact-check-...,,
...,...,...,...,...,...,...,...,...
119,,,,Sponsored New New New,www.youtube.com,https://www.youtube.com/watch?v=2HHqcr43qr0,,
120,,,Biden Is Running For Reelection. What Are His ...,"2024 ELECTION APR. 25, 2023, AT 7:37 AM PHOTO ...",fivethirtyeight.com,https://fivethirtyeight.com/features/biden-ree...,© 2024 ABC News Internet Ventures. All rights ...,
121,,,,,www.cnn.com,https://www.cnn.com/videos/politics/2020/05/22...,© 2024 Cable News Network. A Warner Bros. Disc...,
122,,,Can Joe Biden Intervene in a House Speaker Vot...,As the chaos that has embroiled the House Repu...,www.newsweek.com,https://www.newsweek.com/can-joe-biden-interve...,,None


In [22]:
save_to_csv_with_pandas(df, OUTPUT_FILE)